In [2]:
#!python data/don_char/prepare.py

In [3]:
#%pip install -r "requirements.txt"

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df = pd.read_csv("data/customer_service/train.csv")

# Data Exploration 
Checked the overall information about the dataset

In [48]:
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   issue_area                   970 non-null    object
 1   issue_category               970 non-null    object
 2   issue_sub_category           970 non-null    object
 3   issue_category_sub_category  970 non-null    object
 4   customer_sentiment           970 non-null    object
 5   product_category             970 non-null    object
 6   product_sub_category         970 non-null    object
 7   issue_complexity             970 non-null    object
 8   agent_experience_level       970 non-null    object
 9   agent_experience_level_desc  970 non-null    object
 10  conversation                 970 non-null    object
 11  conversation_length          970 non-null    int64 
dtypes: int64(1), object(11)
memory usage: 91.1+ KB


,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation,conversation_length
count,970,970,970,970,970,970,970,970,970,970,970,970.000000
unique,6,40,109,109,3,3,50,3,3,3,968,NaN
top,Cancellations and returns,Return and Exchange,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Bp Monitor,less,junior,"handles customer inquiries independently, poss...","Agent: You're welcome, Jane. Have a great day!",NaN
freq,277,92,17,17,542,454,26,472,490,490,3,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2164.289691
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572.653613
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1791.250000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2091.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471.750000


Checked the percantages of the different sentiments. It is observed that positive sentiments are very rare.

In [49]:
df["customer_sentiment"].value_counts(normalize=True) * 100

customer_sentiment
neutral     55.876289
negative    42.371134
positive     1.752577
Name: proportion, dtype: float64

Checked the issue complexity vs customer sentiment. It was seen that negative sentiment increases as the issue complexity increases.

In [50]:
pd.crosstab(df["issue_complexity"], df["customer_sentiment"])

customer_sentiment,negative,neutral,positive
issue_complexity,,,
high,39,40,3
less,197,268,7
medium,175,234,7


Similarly, agent experience level vs customer sentiment is examined. Suprisingly, the experience level doesn't have a big affect on the customer sentiment.

In [51]:
pd.crosstab(df["agent_experience_level"], df["customer_sentiment"])

customer_sentiment,negative,neutral,positive
agent_experience_level,,,
experienced,147,188,4
inexperienced,47,93,1
junior,217,261,12


In [52]:
complexity_sentiment = df.groupby("issue_complexity")["customer_sentiment"].value_counts(normalize=True).unstack() * 100
print(complexity_sentiment)
print("\n")
experience_sentiment = df.groupby("agent_experience_level")["customer_sentiment"].value_counts(normalize=True).unstack() * 100
print(experience_sentiment)

customer_sentiment   negative    neutral  positive
issue_complexity                                  
high                47.560976  48.780488  3.658537
less                41.737288  56.779661  1.483051
medium              42.067308  56.250000  1.682692


customer_sentiment       negative    neutral  positive
agent_experience_level                                
experienced             43.362832  55.457227  1.179941
inexperienced           33.333333  65.957447  0.709220
junior                  44.285714  53.265306  2.448980


The lengths of the conversations are inspected. It was seen that the conversations are longer when the customer sentiment is negative.

In [53]:
df["conversation_length"] = df["conversation"].apply(len)
df.groupby("customer_sentiment")["conversation_length"].mean()

customer_sentiment
negative    2433.912409
neutral     1973.374539
positive    1732.588235
Name: conversation_length, dtype: float64

# Data preparation 
Customer service data is preprocessed and saved as .npy files. Here, only the replies given by the users excluding the last 2 replies are taken and saved as X_train and X_val. Also, their corresponding labels (sentiments) are encoded into integer and they are saved as y_train and y_val.

In [55]:
!python data/customer_service/prepare.py

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\user\_netrc
wandb: Currently logged in as: ahmetbekcan to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [60]:
!python train.py config/train_sentiment_analysis.py --compile=False

c:\Users\user\Desktop\DI725-Assignment-1\train.py:215: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.

Overriding config with config/train_sentiment_analysis.py:

  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# train a miniature character-level shakespeare model

Traceback (most recent call last):

...

^^^^^^

x shape: torch.Size([64, 256])

    ...<4 lines>...

y shape: torch.Size([64])

        label_smoothing,
        ^^^^^^^^^^^^^^^^
    )
    ^
ValueError: Expected input batch_size (16384) to match target batch_size (64).
